In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random


from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeCV
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [2]:
# I'm using Colab connected to my Google Drive, so change the path accordingly if you want to run this on local

random.seed(42)

# Import
accepted_full = pd.read_csv("./drive/My Drive/CS156/assignment-2/accepted_2007_to_2018Q4.csv")
rejected_full = pd.read_csv("./drive/My Drive/CS156/assignment-2/rejected_2007_to_2018Q4.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Mapping Rejected -> Accepted

- Policy Code -> policy_code
- Employment Length -> emp_length: turn this into numbers because it's in strings right now
- State -> addr_state
- Zip Code -> zip_code
- Debt-To-Income -> dti
- Risk_Score -> last_fico_range_high and last_fico_range_low
- Application Date -> Not available
- Amount Requested -> loan_amnt

In [3]:
# Data preprocessing

# Step 1: Work with Rejected dataset

# Choose 150000 at random (seeded)
rejected = rejected_full.sample(150000, random_state=42)

# Rename columns
rejected = rejected.drop(labels=["Application Date", "Loan Title", "Zip Code"], axis=1)
rejected_old_cols = list(rejected.columns)
new_col = ["loan_amnt", "risk_score", "dti", "addr_state", "emp_length", "policy_code"]
new_col_mapping = {rejected_old_cols[i]: new_col[i] for i in range(len(new_col))}
rejected = rejected.rename(columns=new_col_mapping)

# Remove rows with nan values
print("Before", rejected.shape)
rejected = rejected.dropna(axis=0)  # Lost a lot of data
print("After", rejected.shape)
rejected.head()

# Convert dti row to float instead of string
def transform_dti_val(val):
    return round(float(val[:-1])/100, 2)

rejected["dti"] = [transform_dti_val(val) for val in list(rejected["dti"])]

# Add funded_amnt column, all 0
rejected["funded_amnt"] = [0.0] * rejected.shape[0]

# Preview
rejected.head()

Before (150000, 6)
After (49169, 6)


,loan_amnt,risk_score,dti,addr_state,emp_length,policy_code,funded_amnt
26499034,7000.0,571.0,0.07,OK,< 1 year,0.0,0.0
9963011,20000.0,626.0,0.32,FL,< 1 year,0.0,0.0
7457258,15000.0,673.0,0.34,NH,< 1 year,0.0,0.0
3751745,10000.0,665.0,0.25,PA,< 1 year,0.0,0.0
14407407,2000.0,601.0,0.01,AZ,< 1 year,0.0,0.0


In [4]:
# Step 2: Work with accepted dataset

# Choose 150000 at random (seeded)
accepted = accepted_full.sample(150000, random_state=42)

# Select relevant columns
accepted = accepted[["loan_amnt", "last_fico_range_high", "last_fico_range_low", "dti", "addr_state", "emp_length", "policy_code", "funded_amnt"]]

# Get risk_score for accepted
accepted["risk_score"] = (accepted["last_fico_range_high"] + accepted["last_fico_range_low"]) / 2
accepted = accepted[["loan_amnt", "funded_amnt", "risk_score", "dti", "addr_state", "emp_length", "policy_code"]]

# Remove rows with nan
print("Before", accepted.shape)
accepted = accepted.dropna(axis=0)  # Lost ~ 10,000 data points, around 6% of the data
print("After", accepted.shape)
accepted.head()

# Turn dti into ratios instead of percentages
accepted["dti"] = [val/100 for val in list(accepted["dti"])]

# Preview
accepted.head()

Before (150000, 7)
After (140263, 7)


,loan_amnt,funded_amnt,risk_score,dti,addr_state,emp_length,policy_code
392949,32000.0,32000.0,792.0,0.2405,CA,10+ years,1.0
324024,4000.0,4000.0,622.0,0.1953,FL,4 years,1.0
2066630,6025.0,6025.0,742.0,0.0916,MA,10+ years,1.0
477199,25000.0,25000.0,657.0,0.3626,CA,10+ years,1.0
1975547,20000.0,20000.0,807.0,0.1643,NV,10+ years,1.0


In [5]:
# Step 3: Make sure the two dataset is balanced before merging

# Because rejected is smaller than accepted, randomly sample the same number of rows from accepted
accepted = accepted.sample(n=rejected.shape[0], random_state=42)

In [6]:
# Step 4: Merge the two datasets and do some encoding for the categorical variables

df = accepted.append(rejected)

# Shuffle the data
df = df.sample(frac = 1, random_state=42)

# Get dummy data for states
df = pd.get_dummies(df, prefix="state_is", columns=["addr_state"])

# Get ordinal encoding for emp_length
mapping = {(str(i) + " years"):i for i in range(2, 10)}
mapping['1 year']  = 1
mapping['< 1 year'] = 0
mapping['10+ years'] = 10
df["emp_length"] = [mapping[val] for val in list(df["emp_length"])]


# Select features and the output variable
X = df.drop("funded_amnt", axis=1)
y = df["funded_amnt"]

In [7]:
# Preview
df.head()

,loan_amnt,funded_amnt,risk_score,dti,emp_length,policy_code,state_is_AK,state_is_AL,state_is_AR,state_is_AZ,state_is_CA,state_is_CO,state_is_CT,state_is_DC,state_is_DE,state_is_FL,state_is_GA,state_is_HI,state_is_IA,state_is_ID,state_is_IL,state_is_IN,state_is_KS,state_is_KY,state_is_LA,state_is_MA,state_is_MD,state_is_ME,state_is_MI,state_is_MN,state_is_MO,state_is_MS,state_is_MT,state_is_NC,state_is_ND,state_is_NE,state_is_NH,state_is_NJ,state_is_NM,state_is_NV,state_is_NY,state_is_OH,state_is_OK,state_is_OR,state_is_PA,state_is_RI,state_is_SC,state_is_SD,state_is_TN,state_is_TX,state_is_UT,state_is_VA,state_is_VT,state_is_WA,state_is_WI,state_is_WV,state_is_WY
8364469,2000.0,0.0,574.0,0.1700,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
26279286,20000.0,0.0,626.0,0.3300,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1358999,30000.0,30000.0,747.0,0.1233,8,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
244138,25000.0,25000.0,737.0,0.0855,9,1.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
999257,15325.0,15325.0,617.0,0.2234,3,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [8]:
# Split the data into training set and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [9]:
high = 1000
regr = RidgeCV(store_cv_values=True, alphas=[x for x in range(1, high)])
regr.fit(X_train, y_train)

# See what the best alpha and score is using CV and Ridge Regression
print("Best Alpha", regr.alpha_)
print("Training set R^2", regr.score(X_train, y_train))

Best Alpha 96
Training set R^2 0.7486664186629699


In [10]:
# Print results for the test set

y_pred = regr.predict(X_test)
print("R^2", regr.score(X_test, y_test))
print("RMSE", (mean_squared_error(y_pred, y_test)**(1/2)))
print("MAE", mean_absolute_error(y_pred, y_test)) # Off by an average amount of 4000 dollars

R^2 0.7492526004307951
RMSE 5084.437634688042
MAE 4046.7810330685975
